In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder

In [3]:
import pandas as pd
import pyodbc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [70]:
server = 'server4project.database.windows.net'
database = 'STERN_DS_Spring2018'
username = 'team4'
password = 'spring2018t5!'
driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+\
                      server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password)
#cursor = cnxn.cursor()

In [71]:
data = pd.read_sql_query("""
SELECT [user_id]
, a.product_id
, add_to_cart_order,order_dow
, order_hour_of_day
, convert(float,days_since_prior_order) as days_since_prior_order
, department, aisle,reordered
FROM [dbo].[order_products__prior] a (nolock)
left join dbo.orders b (nolock) on a.[order_id]=b.[order_id]
left join dbo.[products] c (nolock) on a.product_id = c.product_id
left join dbo.[departments] d (nolock) on c.department_id=d.department_id
left join dbo.[aisles] e (nolock) on c.aisle_id = e.aisle_id
""",cnxn, coerce_float = True)
data['target'] = data.reordered.astype(int)

In [72]:
data['add_to_cart_order'] = data.add_to_cart_order.astype(int)
data['order_dow'] = data.order_dow.astype(int)
data['order_hour_of_day'] = data.order_hour_of_day.astype(int)
data['days_since_prior_order'] = data.days_since_prior_order.astype(int)
data['product_id'] = data.product_id.astype(int)

In [73]:
del data['reordered']

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 9 columns):
user_id                   object
product_id                int32
add_to_cart_order         int32
order_dow                 int32
order_hour_of_day         int32
days_since_prior_order    int32
department                object
aisle                     object
target                    int32
dtypes: int32(6), object(3)
memory usage: 1.4+ GB


In [75]:
data.shape

(32434489, 9)

In [76]:
data.head()

,user_id,product_id,add_to_cart_order,order_dow,order_hour_of_day,days_since_prior_order,department,aisle,target
0,193700,32747,1,6,16,6,dairy eggs,milk,1
1,193700,19508,2,6,16,6,bakery,tortillas flat bread,0
2,193700,26209,3,6,16,6,produce,fresh fruits,1
3,193700,42736,4,6,16,6,dairy eggs,butter,1
4,193700,34243,5,6,16,6,produce,fresh vegetables,0


In [77]:
encoder = OneHotEncoder()
label_encoder = LabelEncoder()
data_label_encoded = label_encoder.fit_transform(data['product_id'])
data['product_id2'] = data_label_encoded

In [ ]:
data_feature_one_hot_encoded = encoder.fit_transform(data[['product_id2']].as_matrix())

In [11]:
#data_feature_one_hot_encoded = encoder.fit_transform(data['product_id2'].values.reshape(-1, 1))
#transformer = LabelBinarizer()
#data_feature_LabelBinarizer = transformer.fit_transform(data[['product_id2']].as_matrix())

In [12]:
#pd.DataFrame(data_feature_one_hot_encoded[:49677].toarray())[0].sum()

In [78]:
data_feature_one_hot_encoded.shape

(32434489, 49677)

In [14]:
transformer1 = LabelBinarizer()
did = pd.DataFrame(transformer1.fit_transform(data['department']), columns=transformer1.classes_)

In [15]:
transformer2 = LabelBinarizer()
aid = pd.DataFrame(transformer2.fit_transform(data['aisle']), columns=transformer2.classes_)

In [16]:
'ff'

'ff'

In [17]:
#concat columns
result = pd.concat([data, did,aid], axis=1)

In [18]:
result.head()

,user_id,product_id,add_to_cart_order,order_dow,order_hour_of_day,days_since_prior_order,department,aisle,target,product_id2,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,193700,32747,1,6,16,6,dairy eggs,milk,1,32741,...,0,0,0,0,0,0,0,0,0,0
1,193700,19508,2,6,16,6,bakery,tortillas flat bread,0,19504,...,0,0,0,1,0,0,0,0,0,0
2,193700,26209,3,6,16,6,produce,fresh fruits,1,26204,...,0,0,0,0,0,0,0,0,0,0
3,193700,42736,4,6,16,6,dairy eggs,butter,1,42728,...,0,0,0,0,0,0,0,0,0,0
4,193700,34243,5,6,16,6,produce,fresh vegetables,0,34237,...,0,0,0,0,0,0,0,0,0,0


In [19]:
del result['user_id']
del result['product_id']
del result['department']
del result['aisle']
del result['product_id2']

In [79]:
data.head()

,user_id,product_id,add_to_cart_order,order_dow,order_hour_of_day,days_since_prior_order,department,aisle,target,product_id2
0,193700,32747,1,6,16,6,dairy eggs,milk,1,32741
1,193700,19508,2,6,16,6,bakery,tortillas flat bread,0,19504
2,193700,26209,3,6,16,6,produce,fresh fruits,1,26204
3,193700,42736,4,6,16,6,dairy eggs,butter,1,42728
4,193700,34243,5,6,16,6,produce,fresh vegetables,0,34237


In [80]:
del data['user_id']
del data['product_id']
del data['department']
del data['aisle']
del data['product_id2']

In [81]:
data.head()

,add_to_cart_order,order_dow,order_hour_of_day,days_since_prior_order,target
0,1,6,16,6,1
1,2,6,16,6,0
2,3,6,16,6,1
3,4,6,16,6,1
4,5,6,16,6,0


In [24]:
#no work memory issue
# transformer = LabelBinarizer()
# pid = pd.DataFrame(transformer.fit_transform(data['product_id2']), columns=transformer.classes_)

In [23]:
#[x for x in result.columns]

In [82]:
data_feature_one_hot_encoded[:7].toarray()[0].shape

(49677,)

In [26]:
# scaler = StandardScaler(copy=True)
# # we have created a standardization based on the training data
# X_clean = scaler.fit(X_mailing_train).transform(X_mailing_train)

In [83]:
# k= np.array([[ 0.,  0.,  0., 0.,  0.,  0.],[6.,6.,6.,6.,6.,6.]])
# j= np.array([[ 1.,  3.,  4., 5.,  6.,  7.],[2.,2.,2.,2.,2.,2.]])
# pd.DataFrame(np.hstack((k,j))).as_matrix()

In [84]:
#np.hstack((k,j))

In [99]:
target = data['target'].as_matrix()

In [100]:
data2 = data.drop(columns=['target']).as_matrix()

In [30]:
did2 = did.as_matrix() 
aid2 = aid.as_matrix() 

In [86]:
print(data2.shape ,
      did2.shape,aid2.shape,data_feature_one_hot_encoded.shape)

(32434489, 5) (32434489, 21) (32434489, 134) (32434489, 49677)


In [97]:
### see all variables
# dir()

In [101]:
data2[:50]

array([[ 1,  6, 16,  6],
       [ 2,  6, 16,  6],
       [ 3,  6, 16,  6],
       [ 4,  6, 16,  6],
       [ 5,  6, 16,  6],
       [ 6,  6, 16,  6],
       [ 7,  6, 16,  6],
       [ 8,  6, 16,  6],
       [ 9,  6, 16,  6],
       [10,  6, 16,  6],
       [ 1,  6, 11, 24],
       [ 2,  6, 11, 24],
       [ 3,  6, 11, 24],
       [ 4,  6, 11, 24],
       [ 1,  0, 21,  8],
       [ 2,  0, 21,  8],
       [ 3,  0, 21,  8],
       [ 4,  0, 21,  8],
       [ 1,  1,  7,  2],
       [ 2,  1,  7,  2],
       [ 3,  1,  7,  2],
       [ 4,  1,  7,  2],
       [ 5,  1,  7,  2],
       [ 6,  1,  7,  2],
       [ 7,  1,  7,  2],
       [ 8,  1,  7,  2],
       [ 1,  1,  9,  0],
       [ 2,  1,  9,  0],
       [ 1,  1, 16,  6],
       [ 2,  1, 16,  6],
       [ 3,  1, 16,  6],
       [ 4,  1, 16,  6],
       [ 5,  1, 16,  6],
       [ 1,  0, 10, 30],
       [ 2,  0, 10, 30],
       [ 3,  0, 10, 30],
       [ 4,  0, 10, 30],
       [ 5,  0, 10, 30],
       [ 6,  0, 10, 30],
       [ 7,  0, 10, 30],


In [105]:
data2.shape

(32434489, 4)

In [90]:
data_feature_one_hot_encoded.shape

(32434489, 49677)

In [91]:
from scipy import sparse

In [102]:
### sparse.hstack because this is a sparsse matrix also needed to clear strings. numerics only
###
#fin = np.hstack( (data2, data_feature_one_hot_encoded ))
fin = sparse.hstack( (data2, data_feature_one_hot_encoded ))
fin = sparse.hstack( (fin, did2))
fin = sparse.hstack( (fin, aid2))

In [106]:
fin.shape

(32434489, 49836)

In [107]:
target

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
logreg = LogisticRegression()

scores = cross_val_score(logreg, fin, target, cv=5)
scores                                              